In [32]:
pwd

'/Users/djb326/Box Sync/me & arrow/pycode/locationspace'

In [226]:
from pathlib import *
import numpy as np
import pandas as pd
from eyepackage.eye_parse import *
from eyepackage.behave_parse import *
from eyepackage.behave_eye_converge import *

In [206]:
%reload_ext autoreload
%autoreload 2

In [136]:
subids=["ec105","ec106","ec107","ec108"]    
pathstring='/Volumes/Voss_Lab/ECOG/ecog/locationspace/ecog.eye/'
eyepath=Path(pathstring)

In [137]:
masternames=get_eye_files(subids,eyepath)

In [138]:
study_all=masternames[masternames['phase']=="a"]
refresh_all=masternames[masternames['phase']=="b"]
recog_all=masternames[masternames['phase']=="c"]

for sub in subids:
    refresh_sub=[]
    refresh_sub=refresh_all[refresh_all['subject']==sub]

In [139]:
# mastereyedf=[]
sub='ec108'
eye_events=parse_eye_line(refresh_sub,pathstring)
#     mastereyedf.append(eyedf)
eyedf=events_to_df(eye_events)
eyearray=eventsdf_cleanup(eyedf)

In [140]:
behavestring='/Volumes/Voss_Lab/ECOG/ecog/locationspace/ecog.behave/'
extra='recogarray.txt'
behaveobj=[behavestring+sub+extra]
behavepath=Path(behaveobj[0])
behavepath.exists()

True

In [141]:
behavearray=read_behave_file(behavepath)
subdict_change_coords=change_behave_coords(subids)

In [142]:
#apply coordinate change to behavioral data if True in subdict
if subdict_change_coords[sub]==True:
    behavearray=apply_adjust_pres_coords(behavearray)
    

In [143]:
subdict_change_coords

{'ec105': False, 'ec106': False, 'ec107': True, 'ec108': True}

In [144]:
reftimesextra='refreshtimes.txt'
reftimesobj=[behavestring+sub+reftimesextra]
reftimespath=Path(reftimesobj[0])
reftimespath.exists()

reftimes=read_times_file(reftimespath)

In [145]:
eyebehave=eye_behave_combo(eyearray,behavearray,reftimes)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [146]:
#adjust timing to object onset
eyebehave=remove_baseline_eye(eyebehave)

In [147]:
# calculate distances for start and end eye locations
startdistarray=calculate_dist(eyebehave,x1='xstart',y1='ystart',name='start')
enddistarray=calculate_dist(eyebehave,'xend','yend','end')

In [148]:
# start locations
eyebehave=loc_view(eyebehave,startdistarray,'startloc')
eyebehave=loc_view(eyebehave,enddistarray,'endloc')

In [149]:
#append start & end distances to eyebehave array
eyebehave=pd.concat([eyebehave, startdistarray, enddistarray], axis=1)

In [207]:
eyebehavedict=eyebehave.to_dict('records')
eyebehavedict=assign_screenview(eyebehavedict,'xstart','ystart','start')
eyebehavedict=assign_screenview(eyebehavedict,'xend','yend','end')

In [227]:
new_previous_events=adjust_fix_before_blink(eyebehavedict)
corrected_eye_events=adjust_event_after_blink(new_previous_events)
subcleandf=eyedict_backto_df(corrected_eye_events)

,block,cond,duration,end,endloc,event,loc1end,loc1start,loc1x,loc1y,...,recog loc,same/diff,start,startloc,sub,trialnum,xend,xstart,yend,ystart
0,1,1.0,32,263.0,loc2end,ESACC,748.743174,608.010016,537.0,943.0,...,1.0,2.0,233.0,loc2start,ec108,1,984.5,980.7,342.7,527.3
1,1,1.0,264,527.0,NaN,EFIX,NaN,746.731009,537.0,943.0,...,1.0,2.0,265.0,loc2start,ec108,1,NaN,969.4,NaN,334.2
3,1,1.0,114,641.0,loc2end,blink,772.787480,733.373472,537.0,943.0,...,1.0,2.0,529.0,loc2start,ec108,1,973.8,941.7,305.5,331.4
4,1,1.0,704,1345.0,NaN,EFIX,NaN,747.221928,537.0,943.0,...,1.0,2.0,643.0,loc2start,ec108,1,NaN,977.6,NaN,339.5
5,1,1.0,54,1399.0,loc3end,ESACC,720.659621,749.438356,537.0,943.0,...,1.0,2.0,1347.0,loc2start,ec108,1,366.5,971.4,242.8,332.3
6,1,1.0,132,1531.0,NaN,EFIX,NaN,718.900501,537.0,943.0,...,1.0,2.0,1401.0,loc3start,ec108,1,NaN,361.7,NaN,245.8
7,1,1.0,22,1553.0,loc3end,ESACC,740.367537,707.401590,537.0,943.0,...,1.0,2.0,1533.0,loc3start,ec108,1,261.5,360.0,255.8,258.1
8,1,1.0,180,1733.0,NaN,EFIX,NaN,747.115480,537.0,943.0,...,1.0,2.0,1555.0,loc3start,ec108,1,NaN,249.3,NaN,253.5
9,1,1.0,62,1795.0,screen,ESACC,263.446997,745.999276,537.0,943.0,...,1.0,2.0,1735.0,loc3start,ec108,1,746.6,254.4,783.4,252.6
10,1,1.0,188,1983.0,NaN,EFIX,NaN,268.974367,537.0,943.0,...,1.0,2.0,1797.0,screen,ec108,1,NaN,750.5,NaN,779.4


In [27]:
l = [i +'hey' for i in etypes]
l

NameError: name 'etypes' is not defined

In [ ]:
type(events)
events[1]

In [ ]:
from mypackage2.mymodule import a_function_i_wrote as f2

In [ ]:
f1()

In [ ]:
f2()

In [ ]:
ls

In [ ]:
cd mypackage2

In [ ]:
ls